# Full_pipeline

**Background:**

**Purpose:**

**Methods:**
>1. Introduction
>2. Inits

**Conclusions:**

# Inits

## Imports

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context(context='notebook', font_scale=1.5)

import logging
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.INFO)

from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.linear_model import LogisticRegression

import mlflow

# Load my own custom module
import data_loading
import constants

import imblearn
import joblib

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import sklearn.metrics as skl_metrics

import joblib

## Definitions

## Funcs

### Generate model wrapper

In [2]:
def holdout_test_wrapper(X, y, test_size=0.25):    
    # Split into CV/test set using target class to stratify
    X_train_untransformed, X_test_untransformed, y_train, y_test = train_test_split(X, 
                                                                                    y, 
                                                                                    test_size=test_size, 
                                                                                    stratify=y)
    
    return prediction_wrapper(X_train_untransformed, y_train, X_test_untransformed, y_test)

### Full prediction wrapper

In [3]:
def prediction_wrapper(X_train_untransformed, y_train, X_test_untransformed, y_test):
    # Create the preprocessing pipeline
    preprocessing_pipeline = Pipeline(steps = [('lower_quantile_cov_removal', remove_lower_CoV_quantiles(q=0.5))])

    # Transform the data
    X_train = preprocessing_pipeline.fit_transform(X_train_untransformed, y_train)
    X_test = preprocessing_pipeline.transform(X_test_untransformed)

    # Fit the model
    log_regr_clf = train_subchallenge_1_model(X_train, y_train)
    
    y_train_pred = log_regr_clf.predict(X_train)
    y_test_pred = log_regr_clf.predict(X_test)
    
    # Create an empty test results dict
    results_dict = {}
    
    # Generate a confusion matrix
    confusion_matrix = skl_metrics.confusion_matrix(y_true=y_test, y_pred=y_test_pred)
    confusion_matrix_dict = {
        'TP': confusion_matrix[1,1],
        'TN': confusion_matrix[0,0],
        'FP': confusion_matrix[0,1],
        'FN': confusion_matrix[1,0]
    }
    
    results_dict['confusion_matrix_dict'] = confusion_matrix_dict
    
    # Calculate train/test accuracy
    results_dict['train_accuracy'] = np.mean(y_train_pred == y_train)
    results_dict['test_accuracy'] = np.mean(y_test_pred == y_test)
    
    # Calculate the train/test AUC
    train_fpr, train_tpr, _ = skl_metrics.roc_curve(y_true=y_train, 
                                                    y_score=log_regr_clf.predict_proba(X_train)[:, 1])
    test_fpr, test_tpr, _ = skl_metrics.roc_curve(y_true=y_test, 
                                                  y_score=log_regr_clf.predict_proba(X_test)[:, 1])
    
    results_dict['train_auc'] = skl_metrics.auc(train_fpr, train_tpr)
    results_dict['test_auc'] = skl_metrics.auc(test_fpr, test_tpr)

    # Calculate the sensitivity and specificity
    results_dict['test_sensitivity'],\
    results_dict['test_specificity'] = calculate_sens_and_spec(y_true=y_test, 
                                                               y_pred=y_test_pred)
    
    # Get the genes with non-zero coefficients
    results_dict['non_zero_gene_list'] = X_train.columns[log_regr_clf.coef_[0] != 0]
    
    return results_dict

### Train sub-challenge 1 model

In [4]:
def train_subchallenge_1_model(X_train, y_train):

    # Define the model
    log_regr_clf = LogisticRegression(penalty='l1',
                                      C=0.129,
#                                      random_state=110,
                                      solver='saga',
                                      max_iter=1e3,
                                      verbose=0,
                                      n_jobs=None,
                                      l1_ratio=None)

    # Run SMOTE on the X_train,y data
    smote_obj = imblearn.over_sampling.SMOTE(sampling_strategy='auto', 
#                                             random_state=110, 
                                             k_neighbors=5, 
                                             n_jobs=None)
    X_train_smote, y_train_smote = smote_obj.fit_sample(X_train, y_train)

    # Train the model
    log_regr_clf.fit(X_train_smote, y_train_smote)
    
    return log_regr_clf

### Remove lower quantile of genes based on CoV

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline 

class remove_lower_CoV_quantiles(BaseEstimator, TransformerMixin):
    def __init__(self, q):
        self.q = q
      
    def fit(self, X, y = None):
        # Calculate the CoV
        gene_expr_cov_list = 100 * (X.std(ddof=1, axis=0) / X.mean(axis=0))

        # Calculate the 0.9 quantile
        min_cov_threshold = np.quantile(gene_expr_cov_list, q=[self.q])[0]

        # Get a list of genes to removed
        self.low_cov_gene_list = gene_expr_cov_list.loc[(gene_expr_cov_list < min_cov_threshold)].index.tolist()
        
        return self
    
    def transform(self, X, y = None):
        return X.drop(self.low_cov_gene_list, axis=1)

### Plot ROC curve

In [6]:
def plot_roc_curve(fpr_list, tpr_list, label=''):
    # Plot the ROC
    plt.plot(fpr_list, tpr_list, label=f'{label} AUC={skl_metrics.auc(fpr_list, tpr_list):0.2f}')
    plt.plot([0,1.0], [0.0, 1.0], 'k--', alpha=0.3)
    
    ax = plt.gca()
    ax.legend()
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    
    return ax

### Calculate Sensitivity and Specificity

In [7]:
def calculate_sens_and_spec(y_true, y_pred):
    # Generate a classification report dict
    classification_report_dict = skl_metrics.classification_report(y_true,
                                                                   y_pred,
                                                                   output_dict=True)
    # Get the sensitivity and specificity from the report dict
    test_sensitivity = classification_report_dict['1']['recall']
    test_specificity = classification_report_dict['0']['recall']

    #print(f'sensitivity={test_sensitivity}, specificity: {test_specificity}')
    
    return test_sensitivity, test_specificity

# Sub-Challenge 1

## Scratch Code

### Run the model

In [ ]:
# Load the gene expression (GE) raw data from file
X, y, phenotype_df = data_loading.load_sc1_data()

# Split into CV/test set using target class to stratify
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [ ]:
# Define the model
log_regr_clf = LogisticRegression(penalty='l1',
                                  C=1e-1,
                                  random_state=110,
                                  solver='saga',
                                  max_iter=1e3,
                                  verbose=0,
                                  n_jobs=None,
                                  l1_ratio=None)

# Remove the lowest quartile of genes based on CoV
# Calculate the CoV
gene_expr_cov_list = 100 * (X_train.std(ddof=1, axis=0) / X_train.mean(axis=0))

# Calculate the 0.9 quantile
min_cov_threshold = np.quantile(gene_expr_cov_list, q=[0.9])[0]

# Get a list of genes to removed
low_cov_gene_list = gene_expr_cov_list.loc[(gene_expr_cov_list < min_cov_threshold)].index.tolist()

# Remove the lowest 0.9 quantile of genes based on CoV
X_train.drop(low_cov_gene_list, axis=1, inplace=True)
X_test.drop(low_cov_gene_list, axis=1, inplace=True)

# Run SMOTE on the X_train,y data
smote_obj = imblearn.over_sampling.SMOTE(sampling_strategy='auto', 
                                         random_state=110, 
                                         k_neighbors=5, 
                                         n_jobs=None)
X_train_smote, y_train_smote = smote_obj.fit_sample(X_train, y_train)

# Train the model
log_regr_clf.fit(X_train_smote, y_train_smote)

# Predict the 

# Log sensitivity, specificity

# Log AUC

#mlflow.sklearn.log_model(svm_clf, "model")

### Calculate AUC

In [ ]:
# Calculate the false positive rate and true positive rate
train_fpr, train_tpr, _ = skl_metrics.roc_curve(y_true=y_train, y_score=log_regr_clf.predict_proba(X_train)[:, 1])
test_fpr, test_tpr, _ = skl_metrics.roc_curve(y_true=y_test, y_score=log_regr_clf.predict_proba(X_test)[:, 1])

plt.figure(figsize=(6,5))
plt.plot(train_fpr, train_tpr, label=f'Train AUC={skl_metrics.auc(train_fpr, train_tpr):0.2f}')
plt.plot(test_fpr, test_tpr, label=f'Test AUC={skl_metrics.auc(test_fpr, test_tpr):0.2f}')
plt.plot([0,1.0], [0.0, 1.0], 'k--', alpha=0.3)

ax = plt.gca()
ax.legend()
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')

### Calculate Sensitivity/Specifity

In [ ]:
classification_report_dict = skl_metrics.classification_report(y_true=y_test,
                                                               y_pred=log_regr_clf.predict(X_test),
                                                               output_dict=True)

test_sensitivity = classification_report_dict['1']['recall']
test_specificity = classification_report_dict['0']['recall']

print(f'sensitivity={test_sensitivity}, specificity: {test_specificity}')

Final function:

In [ ]:
calculate_sens_and_spec(y_true=y_test, 
                        y_pred=log_regr_clf.predict(X_test))

## Final model code

### Scratch code

In [ ]:
# Split into CV/test set using target class to stratify
X_train_untransformed, X_test_untransformed, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

# Create the preprocessing pipeline
preprocessing_pipeline = Pipeline(steps = [('lower_quantile_cov_removal', remove_lower_CoV_quantiles(q=0.5))])

# Transform the data
X_train = preprocessing_pipeline.fit_transform(X_train_untransformed, y_train)
X_test = preprocessing_pipeline.transform(X_test_untransformed)

# Fit the model
log_regr_clf = train_subchallenge_1_model(X_train, y_train)

train_fpr, train_tpr, _ = skl_metrics.roc_curve(y_true=y_train, y_score=log_regr_clf.predict_proba(X_train)[:, 1])
test_fpr, test_tpr, _ = skl_metrics.roc_curve(y_true=y_test, y_score=log_regr_clf.predict_proba(X_test)[:, 1])

# Calculate the train/test AUC
train_auc = skl_metrics.auc(train_fpr, train_tpr)
test_auc = skl_metrics.auc(test_fpr, test_tpr)

# Calculate the sensitivity and specificity
test_sens, test_spec = calculate_sens_and_spec(y_true=y_test, 
                                               y_pred=log_regr_clf.predict(X_test))

In [ ]:
y_test.value_counts()

### Loading data

In [9]:
# Load the gene expression (GE) raw data from file
X, y, phenotype_df = data_loading.load_sc1_data()

### Running holdout tests in parallel

In [ ]:
holdout_test_results_list = joblib.Parallel(n_jobs=-1)\
    (joblib.delayed(holdout_test_wrapper)(X, y) for i in range(24*8))

In [ ]:
test_auc_list = []
test_sens_list = []
test_spec_list = []
for curr_holdout_results_dict in holdout_test_results_list:
    test_auc_list.append(curr_holdout_results_dict['test_auc'])
    test_sens_list.append(curr_holdout_results_dict['test_sensitivity'])
    test_spec_list.append(curr_holdout_results_dict['test_specificity'])

In [ ]:
test_results_df = pd.DataFrame({'test_auc': test_auc_list,
             'test_sens': test_sens_list,
             'test_spec': test_spec_list})

In [ ]:
test_results_df.describe()

### Running a single model

In [10]:
holdout_test_wrapper(X, y, test_size=0.4)

/home/alex/miniconda3/envs/precfda_bc/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


{'confusion_matrix_dict': {'TP': 114, 'TN': 6, 'FP': 14, 'FN': 17},
 'train_accuracy': 0.9203539823008849,
 'test_accuracy': 0.7947019867549668,
 'train_auc': 0.9771712158808933,
 'test_auc': 0.5629770992366412,
 'test_sensitivity': 0.8702290076335878,
 'test_specificity': 0.3,
 'non_zero_gene_list': Index(['ABCB1', 'ABCB7', 'ABHD2', 'ABI3BP', 'ACSBG1', 'ACSL1', 'ACSM5',
        'ACTA1', 'ADAM1A', 'ADCY7',
        ...
        'ZNF544', 'ZNF566', 'ZNF594', 'ZNF649', 'ZNF665', 'ZNF721', 'ZNF738',
        'ZNF785', 'ZNF880', 'ZNF883'],
       dtype='object', length=884)}

In [13]:
10**-0.889

0.12912192736135342

In [11]:
holdout_test_wrapper(X, y, test_size=0.2)

/home/alex/miniconda3/envs/precfda_bc/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


{'confusion_matrix_dict': {'TP': 58, 'TN': 5, 'FP': 5, 'FN': 8},
 'train_accuracy': 0.9269102990033222,
 'test_accuracy': 0.8289473684210527,
 'train_auc': 0.978142589118199,
 'test_auc': 0.6984848484848485,
 'test_sensitivity': 0.8787878787878788,
 'test_specificity': 0.5,
 'non_zero_gene_list': Index(['ABCA8', 'ABCB1', 'ABHD3', 'ABLIM3', 'ACBD5', 'ACO2', 'ACSBG1', 'ACSL3',
        'ACSL6', 'ACSS1',
        ...
        'ZNF605', 'ZNF649', 'ZNF665', 'ZNF721', 'ZNF785', 'ZNF789', 'ZNF83',
        'ZNF880', 'ZNF883', 'ZSWIM9'],
       dtype='object', length=1004)}

In [12]:
holdout_test_wrapper(X, y, test_size=0.1)

/home/alex/miniconda3/envs/precfda_bc/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


{'confusion_matrix_dict': {'TP': 26, 'TN': 2, 'FP': 3, 'FN': 7},
 'train_accuracy': 0.9380530973451328,
 'test_accuracy': 0.7368421052631579,
 'train_auc': 0.9761834100014839,
 'test_auc': 0.5696969696969697,
 'test_sensitivity': 0.7878787878787878,
 'test_specificity': 0.4,
 'non_zero_gene_list': Index(['AASS', 'ABCA5', 'ABCA6', 'ABCA7', 'ABCB1', 'ABCC8', 'ACAP3', 'ACKR3',
        'ACSL1', 'ACTA1',
        ...
        'ZNF680', 'ZNF692', 'ZNF708', 'ZNF721', 'ZNF785', 'ZNF8', 'ZNF83',
        'ZNF880', 'ZNF883', 'ZSWIM9'],
       dtype='object', length=1154)}

In [ ]:
#del X_train
#del X_train_untransformed
del X_test
#del X_test_untransformed
#del y_train
del y_test

In [15]:
log_regr_clf = LogisticRegression(penalty='l1',
                                  C=0.129,
#                                      random_state=110,
                                  solver='saga',
                                  max_iter=1e3,
                                  verbose=0,
                                  n_jobs=None,
                                  l1_ratio=None)

# Run SMOTE on the X_train,y data
smote_obj = imblearn.over_sampling.SMOTE(sampling_strategy='auto', 
#                                             random_state=110, 
                                         k_neighbors=5, 
                                         n_jobs=None)
X_train_smote, y_train_smote = smote_obj.fit_sample(X, y)

# Train the model
log_regr_clf.fit(X_train_smote, y_train_smote)

/home/alex/miniconda3/envs/precfda_bc/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=0.129, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000.0,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [16]:
X.columns[log_regr_clf.coef_[0] != 0]

Index(['AAMDC', 'AASS', 'AATF', 'ABCA5', 'ABCA6', 'ABCA7', 'ABCA8', 'ABCB1',
       'ABCC4', 'ABCC8',
       ...
       'ZNF738', 'ZNF768', 'ZNF785', 'ZNF789', 'ZNF808', 'ZNF83', 'ZNF880',
       'ZNF883', 'ZSWIM7', 'ZSWIM9'],
      dtype='object', length=1837)